In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install joblib
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 19.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cluster import KMeans
import tensorflow as tf
import warnings
import math
from math import sqrt
import sys
import holidays
import datetime

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/training_set_VU_DM.csv', sep=',', nrows=150000)
df.shape

(150000, 54)

In [6]:
# sort values
df = df.sort_values("date_time").reset_index()
df.drop('index',axis=1,inplace=True)
df["date_time"] =  pd.to_datetime(df["date_time"], infer_datetime_format=True)
df["date_time"] = df.date_time.dt.strftime('%Y-%m-%d')
df["date_time_timestamp"] =  pd.to_datetime(df["date_time"], infer_datetime_format=True)
# ax= df[['date_time','rating']].groupby('date_time')['rating'].count().plot(color='blue',figsize=(20,4))
# ax.set_xlabel("Date")
# ax.set_ylabel('Number')
# ax.set_title("Number of Customers Visit")
# plt.show()

<ipython-input-6-e65aa9c04849>:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date_time"] =  pd.to_datetime(df["date_time"], infer_datetime_format=True)
<ipython-input-6-e65aa9c04849>:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date_time_timestamp"] =  pd.to_datetime(df["date_time"], infer_datetime_format=True)


In [7]:
d = datetime.timedelta(days=14)
df['lagged_date_time'] = df["date_time"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d") + d)

In [9]:
df['num_visit']=1
df_test = df.groupby([pd.Grouper(key='lagged_date_time', freq='W-MON')])['num_visit'].count().reset_index().set_index('lagged_date_time').sort_values('lagged_date_time')
df_test=df_test.reset_index()
df_test

,lagged_date_time,num_visit
0,2012-11-19,2828
1,2012-11-26,3635
2,2012-12-03,3912
3,2012-12-10,3539
4,2012-12-17,3648
5,2012-12-24,3722
6,2012-12-31,2919
7,2013-01-07,3713
8,2013-01-14,3814
9,2013-01-21,4041


In [11]:
# def extract_week(feature,week,lag):
#     df[feature] =  pd.to_datetime(df[feature], infer_datetime_format=True)
#     df[feature] = df.date_time.dt.strftime('%Y-%m-%d')
#     if lag == True:
#         d = datetime.timedelta(days=14)
#         df['lag_date_time'] = df[feature].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d") + d)
#         df['week'] = pd.DatetimeIndex(df['lag_date_time']).week
#         df['year']=pd.DatetimeIndex(df['lag_date_time']).year

#         # countinue week numbers for the next year
#         df[week] = df['week'].where(df['year'] ==2013 , df['week']+52)

# extract_week('date_time','click_week',lag=True)

# df.head

def extract_week(feature, week, lag=False):
    if lag:
        # Assuming you're adding a lag of 14 days
        d = datetime.timedelta(days=14)
        df['lag_date_time'] = df[feature] + d
    else:
        df['lag_date_time'] = df[feature]

    df['week'] = df['lag_date_time'].dt.isocalendar().week
    df['year'] = df['lag_date_time'].dt.isocalendar().year

    if lag:
        df[week] = df['week'].where(df['year'] == 2013, df['week'] + 52)

# Example of how to use this function
# Ensure your dataframe 'df' and the 'date_time' column are correctly defined
df['date_time'] = pd.to_datetime(df['date_time'])  # Convert your date column to datetime if not already done
extract_week('date_time', 'click_week', lag=True)

df.head()


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,click_bool,gross_bookings_usd,booking_bool,date_time_timestamp,lagged_date_time,num_visit,lag_date_time,week,year,click_week
0,8415,2012-11-01,5,117,NaN,NaN,220,136962,3,3.5,...,0,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,46,2012,98
1,2137,2012-11-01,5,219,NaN,NaN,219,90248,3,4.0,...,0,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,46,2012,98
2,2137,2012-11-01,5,219,NaN,NaN,219,114001,3,4.0,...,0,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,46,2012,98
3,2137,2012-11-01,5,219,NaN,NaN,219,68228,3,4.5,...,0,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,46,2012,98
4,2137,2012-11-01,5,219,NaN,NaN,219,133124,3,5.0,...,0,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,46,2012,98


In [12]:
# extract month from date_time
df['click_month'] = pd.DatetimeIndex(df['date_time']).month

In [16]:
# Define holidays in some countries
nl_holidays = holidays.Netherlands()
us_holidays = holidays.UnitedStates()

In [37]:
# check if date is in holiday of different countries

df['north_am_ci'] = df['click_week'].apply(lambda x: 1 if x in (us_holidays or nl_holidays)  else 0)

#create_cluster('prop_id')
click_weight = 1
booking_weight = 3
df['rating'] = df['click_bool'] * click_weight + df['booking_bool'] * booking_weight

# print(df)

In [38]:
# df= df.drop(['date_time'],axis=1)
# df= df.drop(['week'],axis=1)
# df= df.drop(['year'],axis=1)
# df= df.drop(['lag_date_time'],axis=1)
# df= df.drop(['date_time_timestamp'],axis=1)
# df= df.drop(['lagged_date_time'],axis=1)
# df= df.drop(['num_visit'],axis=1)
df.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv'

In [43]:
df.to_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_dataset.csv', index=False, encoding='utf-8')

In [25]:
def create_cluster(feature):
    y = df[feature]
    X = df.drop(feature,axis=1)
    wcss=[]
    for i in range(1,11):
        kmeans = KMeans(n_clusters= i, init='k-means++', random_state=0)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1,11), wcss)
    plt.title('The Elbow Method')
    plt.xlabel('number of clusters')
    plt.ylabel('wcss')
    plt.show()

model

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
import warnings

import implicit
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import sys

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [85]:
hotel_train_set = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_dataset.csv', sep=',', nrows=100000)
hotel_train_set.shape


(100000, 61)

In [86]:
hotel_test_set = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/test_set_VU_DM.csv', sep=',', nrows=100000)
hotel_test_set.shape

(100000, 50)

In [45]:
hotel_train_set.head(n=2)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,gross_bookings_usd,booking_bool,date_time_timestamp,lagged_date_time,num_visit,lag_date_time,click_week,click_month,north_am_ci,rating
0,8415,5,117,NaN,NaN,220,136962,3,3.5,0,...,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,98,11,1,0
1,2137,5,219,NaN,NaN,219,90248,3,4.0,1,...,NaN,0,2012-11-01,2012-11-15,1,2012-11-15,98,11,1,0


In [48]:
df = hotel_train_set[['srch_id','prop_id','rating']]
df.shape
# df.head()

,srch_id,prop_id,rating
0,8415,136962,0
1,2137,90248,0
2,2137,114001,0
3,2137,68228,0
4,2137,133124,0


In [90]:
dt = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/test_set_VU_DM.csv', sep=',', nrows=150000)
#create_cluster('prop_id')

dt['rating'] = dt['prop_starrating']

In [92]:
dt.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,rating
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [93]:
df.to_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_testset.csv', index=False, encoding='utf-8')


In [95]:
hotel_test_set = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_testset.csv', sep=',', nrows=100000)

dt = hotel_test_set[['srch_id','prop_id','rating']]

In [49]:
max_rating = df.groupby(['srch_id', 'prop_id']).rating.transform(max)
df = df.loc[df.rating == max_rating]
df.drop_duplicates(keep='first',inplace=True)
df= df.reset_index().drop('index',axis=1)

In [51]:
df.head()

,srch_id,prop_id,rating
0,8415,136962,0
1,2137,90248,0
2,2137,114001,0
3,2137,68228,0
4,2137,133124,0


In [52]:
len(df['srch_id'].unique())

4012

In [53]:
ratings = pd.DataFrame(df.groupby('prop_id')['rating'].mean())
ratings.head()

,rating
prop_id,
1,0.0
3,0.0
4,2.0
9,0.0
11,0.0


In [54]:
ratings['number_ratings'] = pd.DataFrame(df.groupby('prop_id')['rating'].count())
ratings.head()

,rating,number_ratings
prop_id,,
1,0.0,3
3,0.0,2
4,2.0,2
9,0.0,1
11,0.0,2


In [55]:
hotel_matrix = df.pivot_table(index='srch_id',columns='prop_id',values='rating')

In [56]:
hotel_matrix.head()

prop_id,1,3,4,9,11,15,16,17,18,24,...,140788,140795,140799,140800,140801,140808,140809,140813,140815,140816
srch_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
test_matrix = dt.pivot_table(index='srch_id',columns='prop_id',values='rating')
test_matrix.head()

prop_id,1,3,4,9,11,15,16,17,18,24,...,140788,140795,140799,140800,140801,140808,140809,140813,140815,140816
srch_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
def find_similar_clusters(cluster_number):
    #Select user ratings for twohotel_matrixmovies
    item_user_ratings = hotel_matrix[cluster_number]

    # Find correlations between series with corrwith (instead of corr)
    similar_to_hotel = hotel_matrix.corrwith(item_user_ratings)

    # Removing NaN values and using a DataFrame instead of a series
    corr_hotel = pd.DataFrame(similar_to_hotel,columns=['Correlation'])
    corr_hotel.dropna(inplace=True)

    corr_hotel = corr_hotel.join(ratings['number_ratings'])

    result = corr_hotel[corr_hotel['number_ratings']>0].sort_values('Correlation',ascending=False).head()
    return result

In [58]:
warnings.filterwarnings("ignore")
find_similar_clusters(11)

,Correlation,number_ratings
prop_id,,


memory-based CF by computing cosine similarity

In [59]:
n_users = df.srch_id.unique().shape[0]
n_items = df.prop_id.unique().shape[0]
n_users,n_items

(4012, 41820)

In [60]:
list_user = df['srch_id']
list_item = df['prop_id']

In [61]:
# sort dataframe based on user_id and item_id
df = df.sort_values(['srch_id','prop_id'])
df = df.reset_index().drop('index',axis=1)

In [68]:
# apply a function to generate new indexes for user_id and item_id

def id_to_index(df):
    """
    maps the values to the lowest consecutive values
    :param df: pandas Dataframe with columns user, item, rating
    :return: pandas Dataframe with the extra columns index_item and index_user
    """

    index_item = np.arange(0, len(df.prop_id.unique()))
    index_user = np.arange(0, len(df.srch_id.unique()))

    df_item_index = pd.DataFrame(df.prop_id.unique(), columns=["item"])
    df_item_index["new_index"] = index_item
    df_user_index = pd.DataFrame(df.srch_id.unique(), columns=["user"])
    df_user_index["new_index"] = index_user

    df["index_item"] = df["prop_id"].map(df_item_index.set_index('item')["new_index"]).fillna(0)
    df["index_user"] = df["srch_id"].map(df_user_index.set_index('user')["new_index"]).fillna(0)


    return df

In [99]:
df = id_to_index(df)
dt = id_to_index(dt)

In [100]:
# split data to train and test
# train, test = train_test_split(df, test_size=0.3)
# train.shape, test.shape
df.shape, dt.shape


((100000, 5), (100000, 5))

In [103]:
# create a user-item matrix which can be used to calculate the similarity between users and items
data_matrix = np.zeros((n_users, n_items))
for line in df.itertuples():
    data_matrix[line[5], line[4]] = line[3]

train_data_matrix = np.zeros((n_users, n_items))
# unpack the Pandas object
for line in df.itertuples():
    # adjust to count rows and cols from 0 and fill in the matrix
    train_data_matrix[line[5], line[4]] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in dt.itertuples():
    test_data_matrix[line[5], line[4]] = line[3]

In [104]:
train_data_matrix.shape, test_data_matrix.shape

((4012, 41820), (4012, 41820))

In [73]:
# calculating the similarity by using the pairwise_distance from sklearn to calculate the cosine similarity
from sklearn.metrics.pairwise import pairwise_distances

# user-user similarity
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

# item-item similarity
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

user_similarity.shape , item_similarity.shape

((4012, 4012), (41820, 41820))

In [74]:
#make predictions based on these similarities

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [75]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

user_prediction.shape,item_prediction.shape,test_data_matrix.shape

((4012, 41820), (4012, 41820), (4012, 41820))

SVD

In [105]:
sparsity = round(1.0-len(df)/float(n_users * n_items), 3)
print('The sparsity level of MovieLens100K is ' + str(sparsity*100) + '%')


import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix=np.diag(s)

#prediction
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)


The sparsity level of MovieLens100K is 99.9%


In [107]:
import pandas as pd
from scipy.sparse import coo_matrix

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_dataset.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_testset.csv')

# Assuming these dataframes have the columns 'userId', 'itemId', and 'rating'
def create_matrix(df, users, items):
    """ Function to create a user-item sparse matrix """
    return coo_matrix((df['rating'].astype(float), (df['srch_id'], df['prop_id'])),
                      shape=(users, items))

n_users = max(train_df['srch_id'].max(), test_df['srch_id'].max()) + 1
n_items = max(train_df['prop_id'].max(), test_df['prop_id'].max()) + 1

train_matrix = create_matrix(train_df, n_users, n_items).tocsr()
test_matrix = create_matrix(test_df, n_users, n_items).tocsr()


In [108]:
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(mean_squared_error(prediction, ground_truth))


In [109]:
from scipy.sparse.linalg import svds

def evaluate_svd(train, test, k_values):
    best_rmse = float('inf')
    best_k = None

    for k in k_values:
        u, s, vt = svds(train, k=k)
        s_diag_matrix = np.diag(s)
        X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

        current_rmse = rmse(X_pred, test.toarray())  # Convert test matrix to dense format
        print(f'RMSE for k={k}: {current_rmse}')

        if current_rmse < best_rmse:
            best_rmse = current_rmse
            best_k = k

    return best_k, best_rmse

# Possible values for k to test
k_values = [5, 10, 20, 50, 100]
best_k, best_rmse = evaluate_svd(train_matrix, test_matrix, k_values)
print(f'Best k: {best_k} with RMSE: {best_rmse}')


RMSE for k=5: 3.2282321685607016
RMSE for k=10: 3.2163810805874475
RMSE for k=20: 3.1997114116129137
RMSE for k=50: 3.1555098322017954
RMSE for k=100: 3.10898850072142
Best k: 100 with RMSE: 3.10898850072142


In [111]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_dataset.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_testset.csv')

# Function to create a user-item matrix
# def create_matrix(df, n_users, n_items):
#     return coo_matrix((df['rating'], (df['srch_id'], df['prop_id'])), shape=(n_users, n_items)).tocsr()

def create_matrix(df, n_users, n_items):
    return coo_matrix((df['rating'].astype(float), (df['srch_id'], df['prop_id'])),
                      shape=(n_users, n_items),
                      dtype=float).tocsr()

n_users = max(train_df['srch_id'].max(), test_df['srch_id'].max()) + 1
n_items = max(train_df['prop_id'].max(), test_df['prop_id'].max()) + 1

train_matrix = create_matrix(train_df, n_users, n_items)

# # Perform SVD
# u, s, vt = svds(train_matrix, k=20)  # You can tune k based on previous steps
# s_diag_matrix = np.diag(s)
# predictions = np.dot(np.dot(u, s_diag_matrix), vt)

# Check data type and convert if necessary
if train_matrix.dtype != np.float32 and train_matrix.dtype != np.float64:
    train_matrix = train_matrix.astype(float)

# Perform SVD
try:
    if 20 >= min(train_matrix.shape):
        raise ValueError("k must be smaller than the smallest dimension of the matrix")
    u, s, vt = svds(train_matrix, k=20)
    s_diag_matrix = np.diag(s)
    predictions = np.dot(np.dot(u, s_diag_matrix), vt)
except ValueError as e:
    print(e)

def generate_recommendations(predictions, n_recommendations=10):
    recommendations = []
    for user_id in range(predictions.shape[0]):
        user_ratings = predictions[user_id]
        rated_items = train_matrix[user_id].nonzero()[1]

        # Exclude already rated items by setting their prediction to the lowest possible value
        user_ratings[rated_items] = -np.inf

        # Get top-n recommendations
        top_items = user_ratings.argsort()[-n_recommendations:][::-1]
        recommendations.extend([(user_id, item_id) for item_id in top_items])
    return recommendations

recommendations = generate_recommendations(predictions)

# Convert recommendations to DataFrame
recommendation_df = pd.DataFrame(recommendations, columns=['srch_id', 'prop_id'])

# Save to CSV
recommendation_df.to_csv('/content/drive/MyDrive/DMT_g88_assign2/recommendations.csv', index=False)


In [114]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

# Assume functions to create_matrix and compute RMSE are already defined

# Load training and testing data
train_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_dataset.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DMT_g88_assign2/ALS_testset.csv')

# n_users = max(train_df['userId'].max(), test_df['userId'].max()) + 1
# n_items = max(train_df['itemId'].max(), test_df['itemId'].max()) + 1

# # Create user-item matrices
# train_matrix = create_matrix(train_df, n_users, n_items)
# test_matrix = create_matrix(test_df, n_users, n_items)

# Perform SVD on the training data
u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

# Extract test set predictions
test_user, test_item = test_data_matrix.nonzero()
test_predictions = X_pred[test_user, test_item]
test_actual = test_data_matrix[test_user, test_item]
test_actual = test_actual.flatten()

# Calculate RMSE for the test set predictions
print("Test RMSE: ", rmse(test_predictions, test_actual))

# Save predictions
predictions_df = pd.DataFrame({
    'srch_id': test_user,
    'prop_id': test_item,
    'predictedRating': test_predictions
})
predictions_df.to_csv('/content/drive/MyDrive/DMT_g88_assign2/test_predictions.csv', index=False)


Test RMSE:  3.197087923152499


ALS

In [79]:
# Create lists of all users, artists and plays
users = list(np.sort(df.srch_id.unique()))
artists = list(np.sort(df.prop_id.unique()))
plays = list(df.rating)

# Get the rows and columns for our new matrix
rows = df.srch_id.astype(int)
cols = df.prop_id.astype(int)

#data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

In [80]:
train_data_matrix2 = train_data_matrix.nonzero()
train_data_matrix2

(array([   1,    2,    3, ..., 4009, 4010, 4011]),
 array([   59,    63,    85, ..., 10643, 38256, 30315]))

evaluation

In [81]:
user_prediction

array([[-5.38929231e-05,  1.95421462e-04, -5.38929231e-05, ...,
        -5.38929231e-05, -5.38929231e-05, -5.38929231e-05],
       [-2.99645434e-05,  2.19412015e-04, -2.99645434e-05, ...,
        -2.99645434e-05, -2.99645434e-05, -2.99645434e-05],
       [ 4.17789386e-05,  2.91093324e-04,  4.17789386e-05, ...,
         4.17789386e-05,  4.17789386e-05,  4.17789386e-05],
       ...,
       [-2.99749577e-05,  2.19339428e-04, -2.99749577e-05, ...,
        -2.99749577e-05, -2.99749577e-05, -2.99749577e-05],
       [-2.99749577e-05,  2.19339428e-04, -2.99749577e-05, ...,
        -2.99749577e-05, -2.99749577e-05, -2.99749577e-05],
       [ 4.17789386e-05,  2.91093324e-04,  4.17789386e-05, ...,
         4.17789386e-05,  4.17789386e-05,  4.17789386e-05]])

In [82]:
test_data_matrix.shape,user_prediction.shape

((4012, 41820), (4012, 41820))

In [83]:
def rmse(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, y))

def mae(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_absolute_error(prediction, y))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))
print('Model-Based CF RMSE: ' + str(rmse(X_pred, test_data_matrix)))
print("\n")
print('User-based CF MAE: ' + str(mae(user_prediction, test_data_matrix)))
print('Item-based CF MAE: ' + str(mae(item_prediction, test_data_matrix)))
print('Model-Based CF MAE: ' + str(mae(X_pred, test_data_matrix)))

User-based CF RMSE: 3.2482266801595876
Item-based CF RMSE: 3.2483380160271293
Model-Based CF RMSE: 3.248341752022258


User-based CF MAE: 1.705939114671034
Item-based CF MAE: 1.7059706722610901
Model-Based CF MAE: 1.7059732786846946
